# Env
Initiate Environment and Key Parameters below:
- `torrent_dir` -> where you store your torrents, assume it to be where qB stores its torrents
- `content_dir` -> where you store your PT contents. Will ignore hardlinks
- `sorted_torrent_dir` -> where temporary sorted out torrents store.

In [ ]:
import os
import shutil
from difflib import SequenceMatcher
from glob import glob

import torrent_parser as tp

# data = tp.parse_torrent_file('test.torrent')
torrent_dir = r"C:\Users\24103\AppData\Local\qBittorrent\BT_backup"
content_dir = r"P:\MultiMediaDatabase"
sorted_torrent_dir = "result"

# Read Torrent

In [ ]:
torrent_files = glob(torrent_dir + '/*.torrent')
torrent_data = dict()
failed_torrents = list()
for f in torrent_files:
    td = tp.parse_torrent_file(f)
    success = False
    if 'info' in td:
        if 'name' in td['info']:
            torrent_data.update({f: td['info']['name']})
            success = True
    if not success:
        failed_torrents.append(f)

# Read files
`dir_depths_data` -> special dir paths to look into

which means search depths > 1

In [ ]:
dir_depths_data = dict()
for i in os.listdir(content_dir):
    dir_depths_data[i] = [{}]

dir_depths_data.update({'Disc': [{'Animes': [{}]}, {'NonAnimes': [{}]}],
                        'Documents': [{'200MB': [{}]}],
                        'Musics': [{'Animes': [{}]}],
                        'Animes': [{}], 'AnimesTV': [{}], 'Games': [{}], 'NonAnimes': [{}], 'NonAnimesTV': [{}],
                        'R18': [{}],
                        'Softwares': [{}], 'Subtitles': [{}], 'Torrents': [{}],
                        })

def find_files_recursion(dirpath, depth_data: dict) -> dict:
    tmp_filedata = dict()
    for f in os.listdir(dirpath):  # e.g.: Disc
        fullpath = os.path.join(dirpath, f)
        print(f"current: {f}, {fullpath}, {depth_data}")
        if f in depth_data:
            desc_depth_data = depth_data[f]  # e.g.: [{'Animes': []}, {'NonAnimes': []}]
            print(f"desc data: {desc_depth_data}")
            for single_depth_data in desc_depth_data:  # e.g.: {Animes: []}
                print(f"enter recursion: {fullpath}, {single_depth_data}")
                tmp_filedata.update(find_files_recursion(fullpath, single_depth_data))
        else:
            tmp_filedata.update({fullpath: []})
    return tmp_filedata

file_data = find_files_recursion(content_dir, dir_depths_data)

# Match File and Torrent

In [ ]:
match_data = dict()
ratio_threshold = 0.9
for fullpath in file_data:
    for t in torrent_data:
        f = os.path.basename(fullpath)
        tn = torrent_data[t]
        if SequenceMatcher(None, f, tn).quick_ratio() > ratio_threshold:
            print(f"Match: {f} <-> {tn}")
            file_data[fullpath].append(t)

# Copy Torrent and prepare Check

In [ ]:
result_data = dir_depths_data.copy()


def copy_torrent():
    for f, p in file_data.items():
        if len(p):
            torrent_base = os.path.dirname(f)
            torrent_base = torrent_base.replace(content_dir, sorted_torrent_dir)  # rebase
            os.makedirs(torrent_base, exist_ok=True)
            # print(torrent_base)
            for t in p:
                shutil.copy(t, torrent_base)


copy_torrent()

# Auto Reseed

In [ ]:
from qbittorrent import Client

qb = Client('http://127.0.0.1:8080', verify=False)
qb.login('admin', 'admin123')
hash_data = dict()
for d in qb.torrents():
    hash_data[d['hash']] = d

for f, torrents in file_data.items():
    if len(torrents):
        for t in torrents:
            torrent_hash = os.path.splitext(os.path.basename(t))[0]
            if torrent_hash not in hash_data:
                reseed_result = qb.download_from_file(open(t, 'rb'), save_path=os.path.dirname(f), skip_checking=True)
                print(f"reseed: {f} -> {t} = {reseed_result}")

# Auto Tagging

In [ ]:
import qbittorrentapi

qb = qbittorrentapi.Client('localhost', 8080, 'admin', 'admin123')

tags = qb.torrents_tags()
qb.torrents_delete_tags(tags)
tags = []
for h, d in hash_data.items():
    content_path = d['content_path']
    tag = "x" * 100
    while len(tag) > 10:
        content_path = os.path.dirname(content_path)
        tag = os.path.basename(content_path)

    if tag not in tags:
        print(f"Create Tag = {tag}")
        qb.torrents_create_tags(tag)
        tags.append(tag)
    qb.torrents_add_tags(tag, h)

# Other Operations

In [ ]:
find_str = "Ryuu"
for torrent in torrent_data:
    if find_str in torrent_data[torrent]:
        print(f"find torrent {torrent}, {torrent_data[torrent]}")